In [9]:
import FinanceDataReader as fdr

In [10]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import datetime
import shutil
import matplotlib.pyplot as plt

In [125]:
def get_name(symbol, market="KRX"):
    root = "dataset/stock_list"
    
    today = datetime.datetime.now()
    today = datetime.datetime.strftime(today, "%Y_%m_%d")
    
    filename = "stock_list_" + today + ".xlsx"
    save_path = osp.join(root, filename)
    
    if not osp.isfile(save_path):
        if osp.exists(root):
            shutil.rmtree(root)
        if not osp.exists("dataset"):
            os.mkdir("dataset")
        os.mkdir(root)
        df = fdr.StockListing("KRX")
        df.set_index("Symbol", inplace=True)
        name = df.loc["060310", "Name"]
        df.to_excel(save_path)
    
    else:
        df = pd.read_excel(save_path, index_col="Symbol")
        name = df.loc[symbol, "Name"]
        
    return name

In [156]:
def get_exists_excel(symbol, start_date, end_date):
    root = "dataset"
    filename = "_".join([symbol, start_date.replace("-", "_"), end_date.replace("-", "_")]) + ".xlsx"
    save_path = osp.join(root, filename)
    
    if osp.isfile(save_path):
        is_exsists = True
        df = pd.read_excel(save_path, index_col="Date")
        
    else:
        is_exsists = False
        df = None
        
    return is_exsists, df

In [127]:
def save_excel(df, symbol, start_date, end_date):
    if end_date is None:
        end_date = datetime.datetime.now()
        end_date = datetime.datetime.strftime(end_date, "%Y-%m-%d")
        
    root = "dataset"
    if not osp.exists(root):
        os.mkdir(root)
        
    filename = "_".join([symbol, start_date.replace("-", "_"), end_date.replace("-", "_")]) + ".xlsx"
    save_path = osp.join(root, filename)
    df.to_excel(save_path)

In [128]:
def make_day_average_line(df, day=5):
    column = "M" + str(day)
    _df = df.copy()
    _df[column] = _df["Close"].rolling(day).mean()
    return _df

In [218]:
def get_gold_cross_back_testing_by_num(
    symbol:str="005930", 
    start_date:str=None, 
    end_date:str=None, 
    market:str="KRX", 
    num_buy:int=1,
    short:int=5,
    long:int=20,
    show_graph:bool=False,
    fig_save_path:str=None):
    """
        골든 크로스 전략: 
        단기 이평선이 장기 이평선을 아래에서 위로 지나갈 때 매수
        단기 이평선이 장기 이평선을 위에서 아래로 지나갈 때 매도
        
        symbol: 종목번호 혹은 종목티커
        start_date: 조회할 시작 날짜 (입력 안할 시 오늘로부터 30일 이전으로 설정됨)
        start_date: 조회할 끝 날짜 (입력 안할 시 오늘로 설정됨)
        market: KRX (KOSPI, KODAQ, KONEX), NASDAQ, NYSE, AMEX, S&P 500
        num_buy: 한달 매수 수량
        short: 단기 이동평균선 기준 (ex: 5일)
        long: 장기 이동평균선 기준 (ex: 20일)
        show_graph: 그래프 출력 여부
        fig_save_path: 그래프 저장 경로 (확장자명은 생략)
    """
    bought_dict = dict()
    
    if long <= short:
        print("단기 이동평균선 기준은 장기 이동평균선 기준보다 작아야 합니다.")
        return None
    
    name = get_name(symbol, market)
    bought_dict["종목명"] = name
    bought_dict["종목코드"] = symbol
    
    if start_date is None:
        start_date = datetime.datetime.now()
        start_date = start_date - datetime.timedelta(days=30)
        start_date = datetime.datetime.strftime(start_date, "%Y-%m-%d")
        
    if end_date is None:
        end_date = datetime.datetime.now()
        end_date = datetime.datetime.strftime(end_date, "%Y-%m-%d")
        
    is_exsists, df = get_exists_excel(symbol, start_date, end_date)
    if not is_exsists:
        df = fdr.DataReader(symbol, start=start_date, end=end_date)
        save_excel(df, symbol, start_date, end_date)
        
    df = make_day_average_line(df, day=short) # 단기 이평선
    df = make_day_average_line(df, day=long) # 장기 이평선
    
    short_column = "M" + str(short)
    long_column = "M" + str(long)
    
    cond = (df[long_column] < df[short_column]) & (df[long_column].pct_change() > 0)
    df["status"] = np.where(cond, 1, 0)
    buy_cond = (df["status"] == 1) & (df["status"].shift(1) != 1)
    close_cond = df.loc[buy_cond, "Close"]
    
    try:
        current_price = df["Close"].iloc[-1]
        is_data = True
    except IndexError:
        is_data = False
    
    if is_data:
        close_cond.index = \
            [datetime.datetime.strftime(index, "%Y-%m") for index in close_cond.index]
        close_cond = close_cond.groupby(level=0).first()
        close_cond = close_cond.to_frame()
        close_cond["month1"] = [int(index.split("-")[0])*12+int(index.split("-")[1]) for index in close_cond.index]
        close_cond["month2"] = close_cond["month1"].shift(1)
        
        first_index = close_cond.index[0]
        close_cond.loc[first_index, "month2"] = int(start_date[:7].split("-")[0])*12+int(start_date[:7].split("-")[1])
        close_cond["Month"] = close_cond["month1"] - close_cond["month2"]
        close_cond.drop(["month1", "month2"], axis=1, inplace=True)
        close_cond["Num_buy"] = num_buy * close_cond["Month"]
        
        close_cond["Real_price_buy"] = close_cond["Num_buy"]*close_cond["Close"]
        total_num_buy = int(close_cond[["Num_buy"]].sum())
        
        if total_num_buy != 0:
            total_price_buy = int(close_cond[["Real_price_buy"]].sum())
            average_price = total_price_buy//total_num_buy
            bought_dict["매입금액"] = total_price_buy
            bought_dict["보유수량"] = total_num_buy
            bought_dict["현재가"] = current_price
            bought_dict["평균단가"] = average_price
            bought_dict["수익률"] = round((current_price-average_price)/average_price*100, 2)
            bought_dict["평가손익"] = int(total_price_buy * bought_dict["수익률"] / 100)
            
            if show_graph:
                fig = plt.figure(figsize=(12, 8))
                plt.plot(df.index, df["Close"], color="k", label="Close")
                plt.plot(df.index, df[short_column], color="r", label="short_ma")
                plt.plot(df.index, df[long_column], color="b", label="long_ma")
                plt.scatter(new_scatter_data.keys(), new_scatter_data.values(), color="r", marker="o", label="Buy", s=100)
                plt.scatter(not_scatter_data.keys(), not_scatter_data.values(), color="g", marker="^", label="Not Buy", s=100)
                plt.legend()

                if fig_save_path is not None:
                    if not osp.exists("results"):
                        os.mkdir("results")
                    plt.savefig("results/{}.jpg".format(fig_save_path))
                            
            for key, value in bought_dict.items():
                if key in ["종목명", "종목코드"]:
                    continue
                suffix = ""
                prefix = ""

                if key in ["매입금액", "평균단가", "현재가", "평가손익"]:
                    suffix = "원"

                elif key in ["보유수량"]:
                    suffix = "주"

                elif key in ["수익률"]:
                    suffix = "%"

                if key in ["수익률", "평가손익"]:
                    if value > 0:
                        prefix = "+"

                value = prefix+format(value, ",")+suffix
                bought_dict[key] = value
            return bought_dict
        elif total_num_buy == 0:
            print("매수를 하지 않았습니다.")
            bought_dict["수익률"] = "0%"
            return bought_dict   
        
    else:
        print("데이터가 없습니다.")
        bought_dict["수익률"] = "0%"
        return bought_dict

In [220]:
get_gold_cross_back_testing_by_num("005935", "2020-01-01", num_buy=1)

{'종목명': '삼성전자우',
 '종목코드': '005935',
 '매입금액': '485,600원',
 '보유수량': '10주',
 '현재가': '57,400원',
 '평균단가': '48,560원',
 '수익률': '+18.2%',
 '평가손익': '+88,379원'}

In [216]:
def get_gold_cross_back_testing_by_price(
    symbol:str="005930", 
    start_date:str=None, 
    end_date:str=None, 
    market:str="KRX", 
    price_buy:int=50000,
    short:int=5,
    long:int=20,
    show_graph:bool=False,
    fig_save_path:str=None):
    """
        골든 크로스 전략: 
        단기 이평선이 장기 이평선을 아래에서 위로 지나갈 때 매수
        단기 이평선이 장기 이평선을 위에서 아래로 지나갈 때 매도
        
        symbol: 종목번호 혹은 종목티커
        start_date: 조회할 시작 날짜 (입력 안할 시 오늘로부터 30일 이전으로 설정됨)
        start_date: 조회할 끝 날짜 (입력 안할 시 오늘로 설정됨)
        market: KRX (KOSPI, KODAQ, KONEX), NASDAQ, NYSE, AMEX, S&P 500
        price_buy: 한달 매수 수량
        short: 단기 이동평균선 기준 (ex: 5일)
        long: 장기 이동평균선 기준 (ex: 20일)
        show_graph: 그래프 출력 여부
        fig_save_path: 그래프 저장 경로 (확장자명은 생략)
    """
    bought_dict = dict()
    
    if long <= short:
        print("단기 이동평균선 기준은 장기 이동평균선 기준보다 작아야 합니다.")
        return None
    
    name = get_name(symbol, market)
    bought_dict["종목명"] = name
    bought_dict["종목코드"] = symbol
    
    if start_date is None:
        start_date = datetime.datetime.now()
        start_date = start_date - datetime.timedelta(days=30)
        start_date = datetime.datetime.strftime(start_date, "%Y-%m-%d")
        
    if end_date is None:
        end_date = datetime.datetime.now()
        end_date = datetime.datetime.strftime(end_date, "%Y-%m-%d")
        
    is_exsists, df = get_exists_excel(symbol, start_date, end_date)
    if not is_exsists:
        df = fdr.DataReader(symbol, start=start_date, end=end_date)
        save_excel(df, symbol, start_date, end_date)
        
    df = make_day_average_line(df, day=short) # 단기 이평선
    df = make_day_average_line(df, day=long) # 장기 이평선
    
    short_column = "M" + str(short)
    long_column = "M" + str(long)
    
    cond = (df[long_column] < df[short_column]) & (df[long_column].pct_change() > 0)
    df["status"] = np.where(cond, 1, 0)
    buy_cond = (df["status"] == 1) & (df["status"].shift(1) != 1)
    close_cond = df.loc[buy_cond, "Close"]
    
    try:
        current_price = df["Close"].iloc[-1]
        is_data = True
    except IndexError:
        is_data = False
    
    if is_data:
        close_cond.index = \
            [datetime.datetime.strftime(index, "%Y-%m") for index in close_cond.index]
        close_cond = close_cond.groupby(level=0).first()
        close_cond = close_cond.to_frame()
        close_cond["month1"] = [int(index.split("-")[0])*12+int(index.split("-")[1]) for index in close_cond.index]
        close_cond["month2"] = close_cond["month1"].shift(1)
        
        first_index = close_cond.index[0]
        close_cond.loc[first_index, "month2"] = int(start_date[:7].split("-")[0])*12+int(start_date[:7].split("-")[1])
        close_cond["Month"] = close_cond["month1"] - close_cond["month2"]
        close_cond.drop(["month1", "month2"], axis=1, inplace=True)
        close_cond["Price_buy"] = price_buy * close_cond["Month"]
        
        buy_cond = close_cond["Price_buy"] > close_cond["Close"]
        close_cond["Buy"] = np.where(buy_cond, 1, 0)
        close_cond = close_cond[close_cond.Buy != 0]
        close_cond["Num_buy"] = close_cond["Price_buy"]//close_cond["Close"]
        close_cond["Real_price_buy"] = close_cond["Num_buy"]*close_cond["Close"]
        total_num_buy = int(close_cond[["Num_buy"]].sum())
        
        if total_num_buy != 0:
            total_price_buy = int(close_cond[["Real_price_buy"]].sum())
            average_price = total_price_buy//total_num_buy
            bought_dict["매입금액"] = total_price_buy
            bought_dict["보유수량"] = total_num_buy
            bought_dict["현재가"] = current_price
            bought_dict["평균단가"] = average_price
            bought_dict["수익률"] = round((current_price-average_price)/average_price*100, 2)
            bought_dict["평가손익"] = int(total_price_buy * bought_dict["수익률"] / 100)
            
            if show_graph:
                fig = plt.figure(figsize=(12, 8))
                plt.plot(df.index, df["Close"], color="k", label="Close")
                plt.plot(df.index, df[short_column], color="r", label="short_ma")
                plt.plot(df.index, df[long_column], color="b", label="long_ma")
                plt.scatter(new_scatter_data.keys(), new_scatter_data.values(), color="r", marker="o", label="Buy", s=100)
                plt.scatter(not_scatter_data.keys(), not_scatter_data.values(), color="g", marker="^", label="Not Buy", s=100)
                plt.legend()

                if fig_save_path is not None:
                    if not osp.exists("results"):
                        os.mkdir("results")
                    plt.savefig("results/{}.jpg".format(fig_save_path))
                            
            for key, value in bought_dict.items():
                if key in ["종목명", "종목코드"]:
                    continue
                suffix = ""
                prefix = ""

                if key in ["매입금액", "평균단가", "현재가", "평가손익"]:
                    suffix = "원"

                elif key in ["보유수량"]:
                    suffix = "주"

                elif key in ["수익률"]:
                    suffix = "%"

                if key in ["수익률", "평가손익"]:
                    if value > 0:
                        prefix = "+"

                value = prefix+format(value, ",")+suffix
                bought_dict[key] = value
            return bought_dict
        elif total_num_buy == 0:
            print("매수를 하지 않았습니다.")
            bought_dict["수익률"] = "0%"
            return bought_dict   
        
    else:
        print("데이터가 없습니다.")
        bought_dict["수익률"] = "0%"
        return bought_dict

In [217]:
get_gold_cross_back_testing_by_price("005930", "2020-01-01", price_buy=50000)

{'종목명': '삼성전자',
 '종목코드': '005930',
 '매입금액': '261,300원',
 '보유수량': '5주',
 '현재가': '63,200원',
 '평균단가': '52,260원',
 '수익률': '+20.93%',
 '평가손익': '+54,690원'}

### 단기, 장기 이동평균선 변경하면서 비교

In [221]:
index = pd.MultiIndex.from_product([range(5, 10), range(20, 60)])
result = [get_gold_cross_back_testing_by_num("005930", "2019-01-01", short=short, long=long)["수익률"].replace("+", "").replace("%", "") for short, long in index]
s = pd.Series(result, index)
s.sort_values(ascending=False).head() # 최상위 수익률을 보이는 인덱스 5개 보기

5  38     24.6
   37    24.58
   36    23.99
6  38    23.98
7  51    23.81
dtype: object